In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

In [0]:
%sql

select * from delta.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales`

In [0]:
df_src = spark.sql('select distinct(Model_ID),Model_category from delta.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales`')
df_src.display()

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_model'):
    df_sink = spark.sql('select dim_model_key,Model_ID,Model_category from cars_catalog.gold.dim_model')
else:
    df_sink = spark.sql('select 0 dim_model_key,Model_ID,Model_category from delta.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales` where 1=0')


In [0]:
df_filter = df_src.join(df_sink,df_src.Model_ID == df_sink.Model_ID,'left').select(df_src.Model_ID,df_src.Model_category,df_sink.dim_model_key)
df_filter.display()


In [0]:
df_filter_new = df_filter.filter(df_filter.dim_model_key.isNull()).select('Model_ID','Model_category')
df_filter_old = df_filter.filter(df_filter.dim_model_key.isNotNull())


In [0]:
if (incremental_flag == '0'):
    max_value = 1
else:
    if spark.catalog.tableExists('cars_catalog.gold.dim_model'):
        max_value_df = spark.sql('select max(dim_model_key) from  cars_catalog.gold.dim_model')
        max_value = max_value_df.collect()[0][0]
if spark.catalog.tableExists('cars_catalog.gold.dim_model'):
    max_value_df = spark.sql('select max(dim_model_key) from  cars_catalog.gold.dim_model')
    max_value = max_value_df.collect()[0][0]
else:
    max_value = 1

In [0]:
df_filter_new = df_filter_new.withColumn('dim_model_key', max_value+monotonically_increasing_id()+1)


In [0]:
df_final = df_filter_new.union(df_filter_old)

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_model'):
    deltaTable = DeltaTable.forPath(spark,'abfss://gold@azadlsgen7428.dfs.core.windows.net/dim_model')
    deltaTable.alias('t').merge(df_final.alias('s'),'t.dim_model_key = s.dim_model_key').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
    df_final.write.format('delta').mode('append').option('header','true').option('path','abfss://gold@azadlsgen7428.dfs.core.windows.net/dim_model').saveAsTable('cars_catalog.gold.dim_model')